In [3]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import RMSprop,Adam
import os

In [4]:
#超參數
batch_size = 256 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
num_epochs = 20 # 訓練的 epochs 數量

In [6]:
#資料處理
data_generator = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# x_train = next(data_generator.flow(x_train, shuffle=False))

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
from keras.utils import np_utils
# Convert class vectors to binary class matrices.
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [10]:
# 建立 ResNet 模型
from resnet_builder import resnet # 這是從 resnet_builder.py 中直接 import 撰寫好的 resnet 函數
model = resnet(input_shape=(32,32,3)) 
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_4 (Conv2D)              (None, 32, 32, 16)   448         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 16)  64          ['conv2d_4[0][0]']               
 alization)                                                                                       
                                                                                                  
 activation_6 (Activation)      (None, 32, 32, 16)   0           ['batch_normalization[0][0]']

                                                                                                  
 activation_15 (Activation)     (None, 32, 32, 64)   0           ['batch_normalization_9[0][0]']  
                                                                                                  
 conv2d_15 (Conv2D)             (None, 16, 16, 64)   4160        ['activation_15[0][0]']          
                                                                                                  
 batch_normalization_10 (BatchN  (None, 16, 16, 64)  256         ['conv2d_15[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_16 (Activation)     (None, 16, 16, 64)   0           ['batch_normalization_10[0][0]'] 
                                                                                                  
 conv2d_16

                                                                                                  
 conv2d_26 (Conv2D)             (None, 8, 8, 128)    147584      ['activation_25[0][0]']          
                                                                                                  
 batch_normalization_20 (BatchN  (None, 8, 8, 128)   512         ['conv2d_26[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_26 (Activation)     (None, 8, 8, 128)    0           ['batch_normalization_20[0][0]'] 
                                                                                                  
 conv2d_28 (Conv2D)             (None, 8, 8, 256)    33024       ['add_5[0][0]']                  
                                                                                                  
 conv2d_27

In [11]:
batch_size = 64 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 30 # 訓練整個資料集共 30個循環

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/30
782/782 [==============================] - 740s 941ms/step - loss: 1.7958 - accuracy: 0.5250 - val_loss: 2.1710 - val_accuracy: 0.3993
Epoch 2/30
782/782 [==============================] - 776s 992ms/step - loss: 1.3232 - accuracy: 0.6601 - val_loss: 1.5783 - val_accuracy: 0.5709
Epoch 3/30
782/782 [==============================] - 775s 991ms/step - loss: 1.1249 - accuracy: 0.7193 - val_loss: 1.3779 - val_accuracy: 0.6340
Epoch 4/30
782/782 [==============================] - 775s 991ms/step - loss: 0.9898 - accuracy: 0.7635 - val_loss: 1.2456 - val_accuracy: 0.6763
Epoch 5/30
782/782 [==============================] - 768s 982ms/step - loss: 0.8986 - accuracy: 0.7919 - val_loss: 1.3080 - val_accuracy: 0.6660
Epoch 6/30
782/782 [==============================] - 805s 1s/step - loss: 0.8260 - accuracy: 0.8149 - val_loss: 1.0936 - val_accuracy: 0.7327
Epoch 7/30
782/782 [==============================] - 1009s 1s/step - loss: 0.7706 - accuracy: 0.8331 - val_loss: 1.2205 - val_